<a href="https://colab.research.google.com/github/tj-ie/data_analytics/blob/main/school_statics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lifelines
!pip install japanize-matplotlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import BayesianGaussianMixture
from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter

%matplotlib inline

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/sample_data.xlsx')
df

,start,end,sex,age,siblings,distance,duration,payment,address,lat,lng
0,2017-01-29,2019-09-10,0,10,0,10,5,80000,徳島県阿南市長生町西方５８９−１,NaN,NaN
1,2012-01-30,2015-09-11,1,14,1,14,24,400000,徳島県阿南市下大野町渡り上り５４４,NaN,NaN
2,2019-01-31,2021-05-10,1,13,2,23,30,480000,徳島県阿南市羽ノ浦町岩脇町筋８７,NaN,NaN
3,2016-02-01,2017-09-13,0,14,0,8,15,240000,徳島県阿南市羽ノ浦町中庄大久保３３−３,NaN,NaN
4,2017-02-02,2019-09-14,1,17,2,5,10,160000,徳島県阿南市那賀川町今津浦白石１２−５,NaN,NaN
5,2017-02-03,2021-05-10,1,15,1,11,18,280000,徳島県阿南市那賀川町色ケ島民養,NaN,NaN
6,2020-02-04,2020-09-16,0,11,0,3,15,260000,徳島県阿南市那賀川町今津浦喜来６５−１,NaN,NaN
7,2020-02-04,2020-09-16,0,30,0,12,9,140000,徳島県阿南市那賀川町今津浦喜来sadf,NaN,NaN


In [ ]:
df.isnull().sum()

start       0
end         0
sex         0
age         0
siblings    0
distance    0
duration    0
payment     0
address     0
lat         8
lng         8
dtype: int64

In [ ]:
for _ in range(2):
  df = df.drop(df['age'].idxmax())
  df = df.reset_index()

In [ ]:
df

,level_0,index,start,end,sex,age,siblings,distance,duration,payment,address,lat,lng
0,0,0,2017-01-29,2019-09-10,0,10,0,10,5,80000,徳島県阿南市長生町西方５８９−１,NaN,NaN
1,1,1,2012-01-30,2015-09-11,1,14,1,14,24,400000,徳島県阿南市下大野町渡り上り５４４,NaN,NaN
2,2,2,2019-01-31,2021-05-10,1,13,2,23,30,480000,徳島県阿南市羽ノ浦町岩脇町筋８７,NaN,NaN
3,3,3,2016-02-01,2017-09-13,0,14,0,8,15,240000,徳島県阿南市羽ノ浦町中庄大久保３３−３,NaN,NaN
4,5,5,2017-02-03,2021-05-10,1,15,1,11,18,280000,徳島県阿南市那賀川町色ケ島民養,NaN,NaN
5,6,6,2020-02-04,2020-09-16,0,11,0,3,15,260000,徳島県阿南市那賀川町今津浦喜来６５−１,NaN,NaN


In [ ]:
import requests
import time
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
url = 'http://www.geocoding.jp/api/'

for i, row in df.iterrows():
  payload = {'q': row['address']}
  html = requests.get(url, params=payload)
  soup = BeautifulSoup(html.content, 'html.parser')
  if soup.find('error'):
    raise ValueError(f'Invalid address submitted. {i}{row["address"]}')
  else:
    # 緯度経度をdfに追加
    print(html.content)
    print(payload)
    lat = soup.find('lat').string
    lng = soup.find('lng').string

    print(lat)
    print(lng)
    df.loc[i, row['lat']] = lat
    df.loc[i, row['lng']] = lng
    time.sleep(10)

b'HTTP Error 403 Forbidden.'
{'q': '徳島県阿南市長生町西方５８９−１'}


AttributeError: ignored

In [ ]:
df

,level_0,index,start,end,sex,age,siblings,distance,duration,payment,address,lat,lng,NaN
0,0,0,2017-01-29,2019-09-10,0,10,0,10,5,80000,徳島県阿南市長生町西方５８９−１,NaN,NaN,NaN
1,1,1,2012-01-30,2015-09-11,1,14,1,14,24,400000,徳島県阿南市下大野町渡り上り５４４,NaN,NaN,NaN
2,2,2,2019-01-31,2021-05-10,1,13,2,23,30,480000,徳島県阿南市羽ノ浦町岩脇町筋８７,NaN,NaN,NaN
3,3,3,2016-02-01,2017-09-13,0,14,0,8,15,240000,徳島県阿南市羽ノ浦町中庄大久保３３−３,NaN,NaN,NaN
4,5,5,2017-02-03,2021-05-10,1,15,1,11,18,280000,徳島県阿南市那賀川町色ケ島民養,NaN,NaN,NaN
5,6,6,2020-02-04,2020-09-16,0,11,0,3,15,260000,徳島県阿南市那賀川町今津浦喜来６５−１,NaN,NaN,NaN
